In [245]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [246]:
txt = 'good product. higly recommended.'

In [247]:
#Remove punctuations ('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text


In [248]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()


In [249]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [250]:
def preprocessing(text):
    data = pd.DataFrame([text],columns=['tweet'])
    # convert uppercase to lowercase
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    #Remove Numbers
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    #Remove stop word
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
    #remove stem
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(ps.stem(x) for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    
    return data['tweet']

In [251]:
preprocessed_txt = preprocessing(txt)

In [252]:
txt

'good product. higly recommended.'

In [253]:
preprocessed_txt

0    good product higli recommended
Name: tweet, dtype: object

In [254]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [255]:
tokens[:5]

['test', 'android', 'app', 'beauti', 'cute']

In [256]:
# vectorization

def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.flot32)
    return vectorized_lst_new
        

In [257]:
vectorized_txt = vectorizer(preprocessing_txt,tokens)

In [258]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Import Model

In [259]:
with open('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

In [260]:
model.predict(vectorized_txt)

array([1], dtype=int64)

# Summary of Prediction Pipeline

In [292]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [293]:
#Remove punctuations ('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text


In [294]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [295]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [296]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [297]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [298]:
def preprocessing(text):
    data = pd.DataFrame([text],columns=['tweet'])
    # convert uppercase to lowercase
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    #Remove Numbers
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    #Remove stop word
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
    #remove stem
    data['tweet'] = data['tweet'].apply(lambda x:" ".join(ps.stem(x) for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    
    return data['tweet']

In [301]:
# vectorization

def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.float32)
    return vectorized_lst_new
        

In [310]:
def get_prediction(vectorized_text):
    predction = model.predict(vectorized_text)
    if predction == 1:
        return 'Negative'
    else:
        return 'Positive'

In [311]:
txt = "awesome product,. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt,tokens)
predction = get_prediction(vectorized_txt)
predction

'Positive'